In [ ]:
import os
import sys
import data
import numpy             as np
import pandas            as pd
import scipy.io          as sio
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.size'] = 14

In [ ]:
patt      = '*Inpatient_Claims_Sample_*' # regex
data_path = os.path.join(os.path.realpath('.'), 'synthetic_data/')

# regex pattern for file type (we are focusing on Inpatient for our analysis)
df       = data.read_in_all_files(data_path, patt)
model_df = data.create_inpatient_core_df(df)
model_df = data.add_summary_info(model_df, data_path)

In [ ]:
df_in_amt  = model_df['clm_pmt_amt'       ].values
df_in_days = model_df['clm_utlztn_day_cnt'].values
# list(model_df.columns.values)

In [ ]:
## Select features
# Case 1: x1 = claim utilization day counts; x2 = claim payment amount
# df_in_days = df_in.CLM_UTLZTN_DAY_CNT.to_numpy()
# df_in_amt  = df_in.CLM_PMT_AMT.to_numpy()
X = np.array([df_in_days, df_in_amt]).T

# Set empty entries to k days
k = 1
Idx_nan = np.argwhere(np.isnan(X))
X [Idx_nan, 0] = k

plt.figure()
plt.scatter(X[:, 0], X[:, 1])
plt.xlabel('Claim utilization days (day)')
plt.ylabel('Claim payment amount (dollar)')

In [ ]:
#Estimate the dataset statistics
#Assume a Gaussian distribution
def estimateGaussian(X):
    m, n   = X.shape
    mu     = np.mean(X, 0)
    sigma2 = np.var(X, 0)
    return mu, sigma2

In [ ]:
mu, sigma2 = estimateGaussian(X)

In [ ]:
def multivariateGaussian(X, mu, sigma2):
    k = mu.size
    if (sigma2.ndim==1):
        sigma2 = np.diag(sigma2)
    X = X - mu
    p = (2*np.pi)**(-k/2)*np.linalg.det(sigma2)**(-0.5)*np.exp(-0.5*np.sum(X.dot(np.linalg.pinv(sigma2))*X, 1))
    return p   

In [ ]:
#visualize Gaussian fit
def visualizeFit(X, mu, sigma2):
    x1 = np.linspace(0, 35, 71)
    x2 = np.linspace(0, 35, 71)
    xv1, xv2 = np.meshgrid(x1, x2)
    Xv = np.array([xv1.ravel(), xv2.ravel()])
    Z = multivariateGaussian(Xv.T, mu, sigma2)
    Z = np.reshape(Z, xv1.shape)
    plt.figure()
    plt.scatter(X[:, 0], X[:, 1])
    exp = np.linspace(-20, 0, 6)
    plt.contour(xv1, xv2, Z, 10**exp.T)

In [ ]:
# def selectThreshold(yval, pval):
#     bestEpsilon = 0.
#     bestF1 = 0.
#     F1 = 0.
#     stepsize = (np.max(pval) - np.min(pval)) / 1000.
#     for epsilon in np.arange(np.min(pval), np.max(pval), stepsize):
#         predictions = pval < epsilon
#         tp = np.sum((predictions==1) & (yval.T==1)) * 1.0
#         fp = np.sum((predictions==1) & (yval.T==0)) * 1.0
#         fn = np.sum((predictions==0) & (yval.T==1)) * 1.0
#         prec = tp/(tp+fp)              if tp + fp    > 0.0 else 0.0
#         rec  = tp/(tp+fn)              if tp + fn    > 0.0 else 0.0
#         F1   = (2*prec*rec)/(prec+rec) if prec + rec > 0.0 else 0.0
#         if F1 > bestF1:
#             bestF1 = F1
#             bestEpsilon = epsilon
#         f1_log.append(F1)
#         eps_log.append(epsilon)
#     return bestEpsilon, bestF1

In [ ]:
# global f1_log, eps_log
# f1_log  = []
# eps_log = []

# #select threshold based on F1 score on cross validation set
# pval        = multivariateGaussian(Xval, mu, sigma2)
# epsilon, F1 = selectThreshold(yval, pval)
# epsilon
#plt.plot(eps_log, f1_log)

In [ ]:
# Specify threshold probability epsilon
epsilon = 1e-10

In [ ]:
#Find the outliers
p        = multivariateGaussian(X, mu, sigma2)
outliers = np.argwhere(p<epsilon)
visualizeFit(X, mu, sigma2)
plt.scatter(X[outliers, 0], X[outliers, 1])

In [ ]:
plt.hist(X[:,1], bins=30, normed=True)

In [ ]:
plt.hist(X[:,0], bins=30, normed=True)